In [1]:
%%html
<style>
.cell-output-ipywidget-background {
   background-color: transparent !important;
}
.jp-OutputArea-output {
   background-color: transparent;
}  
</style>

In [2]:
import numpy as np
from time import sleep, time
from threading import Thread
from ipycanvas import Canvas, hold_canvas
from ipywidgets import Output

out = Output()

In [12]:
class Network:
    def __init__(self, nonlinearity=5):
        self.memories = []
        self.nonlinearity = nonlinearity
    
    def add_memory(self, memory):
        self.memories.append(memory)
    
    def calculate_weights(self):
        weights = []
        for memory in self.memories:
            _weights = np.outer(memory, memory)
            _weights /= np.mean(memory @ _weights) / np.mean(memory)
            weights.append(_weights)
        self.weights = np.mean(weights, axis=0)

    def single_value_hopfield_update(self, state, index):
        # single value hopfied update
        value = self.weights[index] @ state
        state[index] = np.clip(self.nonlinearity * value, -1, 1)
        return state

# # hopfield update
# state = weights @ state
# # normalize
# state = state / np.average(state)

In [4]:
def draw(state, canvas, n_pixels=50):
    state = (state + 1) / 2
    with hold_canvas(canvas):
        canvas.clear()
        state = state.reshape(8, 8)
        rects = []
        for i, row in enumerate(state):
            for j, value in enumerate(row):
                # canvas.fill_style = f'rgb({int(value * 255)}, {int(value * 255)}, {int(value * 255)})'
                # canvas.fill_rect(i * n_pixels, j * n_pixels, n_pixels, n_pixels)
                value = int(value * 255)
                # color = f'rgb({value}, {value}, {value})'
                color = (value, value, value)
                rects.append((i * n_pixels, j * n_pixels, n_pixels, n_pixels, color, 1))
        
        xs, ys, ws, hs, colors, alphas = zip(*rects)
        xs, ys, ws, hs, colors, alphas = list(xs), list(ys), list(ws), list(hs), list(colors), list(alphas)
        canvas.fill_styled_rects(xs, ys, ws, hs, colors, alphas)

In [5]:
n_pixels = 50

canvas = Canvas(width=8 * n_pixels, height=8 * n_pixels)
network = Network()

# black state
state = -np.ones(64)
draw(state, canvas)

# drawing on canvas with mouse
@out.capture()
def handle_mouse_down(x, y):
    x = int(x / n_pixels)
    y = int(y / n_pixels)
    print(x, y)
    index = x * 8 + y
    if state[index] != 1.0:
        state[index] = 1.0
        draw(state, canvas)

canvas.on_mouse_down(handle_mouse_down)
canvas

Canvas(height=400, width=400)

In [9]:
network.add_memory(state)
network.calculate_weights()

# black state
state = -np.ones(64)
draw(state, canvas)

In [43]:
# initialize random state
state = np.random.choice([-1.0, 1.0], size=64)
draw(state, canvas)

In [44]:
# randomply shuffled indexes
indexes = np.arange(0, 64)
np.random.shuffle(indexes)
# duplicate indexes
# indexes = np.concatenate([indexes, indexes])

start_time = time()
for i, index in enumerate(indexes):
    _to_sleep = i * 0.01 - (time() - start_time)
    sleep(max(0, _to_sleep))

    draw(state, canvas)
    state = network.single_value_hopfield_update(state, index)

In [13]:
network.nonlinearity = 5